## UAV-Cyber-SIM

In [1]:
import random
from collections import defaultdict

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Create Plans

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs=[Color.GREEN,Color.ORANGE,Color.RED,Color.BLUE]
n_uavs_per_gcs = 60
side_len = 10
altitude = 5
max_delay = 10 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len-1_500, 0, 0)
    for i in range(len(gcs))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids: dict[str, list[int]] = defaultdict(list)

for i,color in enumerate(colors,start=1):
    gcs_sysids[f'{color.name} {color.emoji}'].append(i)

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

# No Visualizer

In [5]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Visualization Parameters

In [6]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulator

In [7]:
simulator = Simulator(
	visualizers=[novis],
	gcs_sysids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

🚀 ArduPilot SITL vehicle 1 launched (PID 2604961)
🚀 ArduPilot SITL vehicle 3 launched (PID 2604967)
🚀 UAV logic for vehicle 1 launched (PID 2604962)
🚀 Proxy for vehicle 1 launched (PID 2604970)
🚀 ArduPilot SITL vehicle 2 launched (PID 2604965)
🚀 UAV logic for vehicle 2 launched (PID 2604977)
🚀 ArduPilot SITL vehicle 5 launched (PID 2604981)
🚀 Proxy for vehicle 2 launched (PID 2604980)
🚀 UAV logic for vehicle 3 launched (PID 2604969)
🚀 ArduPilot SITL vehicle 4 launched (PID 2604976)
🚀 Proxy for vehicle 3 launched (PID 2604984)
🚀 UAV logic for vehicle 4 launched (PID 2604988)
🚀 UAV logic for vehicle 5 launched (PID 2604982)
🚀 ArduPilot SITL vehicle 7 launched (PID 2604995)
🚀 ArduPilot SITL vehicle 6 launched (PID 2604990)
🚀 UAV logic for vehicle 7 launched (PID 2605100)
🚀 Proxy for vehicle 7 launched (PID 2605107)
🚀 Proxy for vehicle 5 launched (PID 2605099)
🚀 UAV logic for vehicle 6 launched (PID 2605101)
🚀 ArduPilot SITL vehicle 9 launched (PID 2605105)
🚀 ArduPilot SITL vehicle 8 launc

🚀 UAV logic for vehicle 8 launched (PID 2605117)
🚀 Proxy for vehicle 9 launched (PID 2605127)
🚀 ArduPilot SITL vehicle 11 launched (PID 2605126)
🚀 UAV logic for vehicle 10 launched (PID 2605124)
🚀 ArduPilot SITL vehicle 12 launched (PID 2605130)
🚀 ArduPilot SITL vehicle 13 launched (PID 2605182)
🚀 Proxy for vehicle 8 launched (PID 2605183)
🚀 ArduPilot SITL vehicle 14 launched (PID 2605188)
🚀 ArduPilot SITL vehicle 19 launched (PID 2605295)
🚀 Proxy for vehicle 10 launched (PID 2605192)
🚀 UAV logic for vehicle 11 launched (PID 2605186)
🚀 UAV logic for vehicle 12 launched (PID 2605193)
🚀 ArduPilot SITL vehicle 15 launched (PID 2605196)
🚀 ArduPilot SITL vehicle 18 launched (PID 2605205)
🚀 ArduPilot SITL vehicle 16 launched (PID 2605200)
🚀 ArduPilot SITL vehicle 17 launched (PID 2605203)
🚀 ArduPilot SITL vehicle 20 launched (PID 2605353)
🚀 UAV logic for vehicle 14 launched (PID 2605351)
🚀 ArduPilot SITL vehicle 22 launched (PID 2605409)
🚀 ArduPilot SITL vehicle 21 launched (PID 2605406)
🚀 U

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 227: ✅ Sent mission item 7
Vehicle 227: ▶️ Step Started: load mission /home/abeldg/uav-cyber-sim/data/mission_227.waypoints
Vehicle 227: ✅ Mission upload successful!
Vehicle 227: ✅ Step Done: load mission /home/abeldg/uav-cyber-sim/data/mission_227.waypoints
Vehicle 227: ✅ Action Done: 📤 UPLOAD_MISSION
Vehicle 225: ✅ Sent mission item 0
Vehicle 235: ✅ Step Done: clear uav missions
Vehicle 227: ▶️ Action Started: 🔧 PREARM
Vehicle 225: ✅ Sent mission item 1
Vehicle 227: ▶️ Step Started: Check disarmed
Vehicle 235: ✅ 8 waypoints read.
Vehicle 225: ✅ Sent mission item 2
Vehicle 231: ✅ Step Done: clear uav missions
Vehicle 225: ✅ Sent mission item 3
Vehicle 231: ✅ 8 waypoints read.
Vehicle 240: ✅ Step Done: clear uav missions
Vehicle 225: ✅ Sent mission item 4
Vehicle 240: ✅ 8 waypoints read.
Vehicle 225: ✅ Sent mission item 5
Vehicle 225: ✅ Sent mission item 6
Vehicle 223: ✅ Step Done: Check disarmed
Vehicle 225: ✅ Sent mission item 7
Vehicle 225: ▶️ Step Started: load mission /hom